In [8]:
using DrWatson

In [9]:
@quickactivate "RxInferThesisExperiments"

In [15]:
using RxInferThesisExperiments, Turing, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, ProgressMeter

In [30]:
const bfolder_nuts = datadir("hgf", "turing", "nuts")

"/Users/bvdmitri/.julia/dev/thesis/data/hgf/turing/nuts"

In [32]:
# Pregenerate paths for benchmark data
mkpath(bfolder_nuts);

In [33]:
const environment = HGFEnvironment()

HGFEnvironment()

In [38]:
# Include the model specification
include(srcdir("models", "turing", "hgf.jl"));

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


In [43]:
function run_benchmark_nuts(params)
    @unpack T, nsamples, seed = params
    
    zstates, xstates, observations = rand(StableRNG(seed), environment, T);
    model    = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
        HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
    end
    result   = run_inference(model, observations; nsamples = nsamples, rng = StableRNG(seed))
    e_states = extract_posteriors(T, result)
    z_amse   = compute_amse(zstates, e_states[:z])
    x_amse   = compute_amse(xstates, e_states[:x])
    
    # Turing need to recreate the model every time....
    benchmark_modelcreation = @benchmark begin end
    
    benchmark_inference = @benchmark run_inference(model, observations; nsamples = $nsamples, rng = StableRNG($seed)) setup=begin
        model = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
            HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
        end
        zstates, xstates, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local zstates, xstates, observations = rand(StableRNG(_seed), environment, T);
        local model = (observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior) -> begin 
            HGF(observation, zt_min_prior, xt_min_prior, z_std_prior, y_std_prior, environment.kappa, environment.omega)
        end
        local result   = run_inference(model, observations; nsamples = nsamples, rng = StableRNG(_seed))
        local e_states = extract_posteriors(T, result)
        return [ compute_amse(zstates, e_states[:z]), compute_amse(xstates, e_states[:x]) ]
    end
    
    z_emse = emse[1]
    x_emse = emse[2]
    
    states = (z = zstates, x = xstates)
    
    output = @strdict T nsamples seed states e_states observations z_amse x_amse z_emse x_emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark_nuts (generic function with 1 method)

In [44]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params_nuts = dict_list(Dict(
    "T"           => [ 10, 20 ],
    "nsamples"    => [ 100, 200 ],
    "seed"        => [ 42 ]
));

In [45]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks_nuts = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark_nuts, bfolder_nuts, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/hgf/turing/nuts/T=10_nsamples=100_seed=42.jld2 does not exist. Producing it now...
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
P

In [46]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/hgf/turing/nuts for result files.
[ Info: Added 4 entries.


Row,T,nsamples,z_emse,x_emse,z_amse,seed,x_amse,inference,creation
,Int64?,Int64?,Float64?,Float64?,Float64?,Int64?,Float64?,Tuple…?,Tuple…?
1,10,100,0.415509,2.89447,0.0754081,42,3.07153,"(9.14985e9, 9.14985e9, 8.50423e8)","(1.723, 1.80874, 0.0)"
2,10,200,0.994373,3.23911,0.693759,42,3.05412,"(1.71935e10, 1.71935e10, 1.58064e9)","(1.611, 1.75376, 0.0)"
3,20,100,0.395771,5.40503,0.0754647,42,10.0394,"(1.73864e10, 1.73864e10, 1.59673e9)","(1.612, 1.77466, 0.0)"
4,20,200,0.690889,4.92082,0.442536,42,8.81883,"(3.54283e10, 3.54283e10, 3.38062e9)","(1.612, 1.75244, 0.0)"


# Versions

In [47]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 12 virtual cores


In [48]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/thesis/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.4
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.0
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.0
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1
  [37e2e46d] Line